# Formatting protected areas shapefiles

Creating 4 files:
 - terras indigenas (`TI`): only indigenous lands
 - terras indigenas and protecao integral (`TI_PI`): indigenous land and complete protection
 - uso sustentavel (`US`): sustainable use
 - all categories together

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.sparse.csgraph import connected_components

In [2]:
# Function that dissolves into unique polygon all those polygons overlapping each other
def dissolve_interesct(data):
    # Matrix indicating the geometries intersecting each other
    overlap_matrix = data.geometry.apply(lambda x: data.geometry.intersects(x)).values.astype(int)
    
    # Get the groups of connected geometries
    n, ids = connected_components(overlap_matrix)
    
    # Dissolve the geometries into unique ones based on these groups
    # Create GeodataFrame indicating the group each polygon belongs to
    df = gpd.GeoDataFrame({'groups': ids, 'geometry': data.geometry}, crs = data.crs)
    # Dissolve
    df = df.dissolve(by = 'groups').reset_index()
    
    return df

In [3]:
# Function that calculates the area of each polygon in km2
def calculateArea_km2(data):
    
    # Record original CRS
    or_crs = data.crs
    
    # Convert to metered CRS
    data = data.to_crs('epsg:5880')
    
    # Calculate area
    data['area_km2'] = data.geometry.area * 1e-06
    
    # Convert CRS back
    data = data.to_crs(or_crs)
    
    return data

In [4]:
cerrado = gpd.read_file('../../data/shapes/cerrado_1-250000.shp')
cerrado = cerrado.to_crs('epsg:4674')

## Terras indigenas

In [5]:
ti = gpd.read_file('D:/carlota/projects/data/terras_indigenas_FUNAI/ti_sirgasPolygon.shp')

In [6]:
ti.head()

,gid,terrai_cod,terrai_nom,etnia_nome,municipio_,uf_sigla,superficie,fase_ti,modalidade,reestudo_t,cr,faixa_fron,undadm_cod,undadm_nom,undadm_sig,geometry
0,1,101,Acapuri de Cima,Kokama,Fonte Boa,AM,18393.9411,Declarada,Tradicionalmente ocupada,None,COORDENACAO REGIONAL DO ALTO SOLIMOES,Não,3.020200e+10,COORDENACAO REGIONAL DO ALTO SOLIMOES,CR-AS,"POLYGON ((-66.88359 -2.53752, -66.87457 -2.537..."
1,2,201,Acimã,Apurinã,Lábrea,AM,40686.0340,Regularizada,Tradicionalmente ocupada,None,COORDENACAO REGIONAL MÉDIO PURUS,Não,3.020200e+10,COORDENACAO REGIONAL MÉDIO PURUS,CR-MPUR,"POLYGON ((-66.30929 -7.79995, -66.30929 -7.799..."
2,57,601,Aconã,Tingui-Botó,Traipu,AL,267.7862,Regularizada,Reserva Indígena,None,COORDENACAO REGIONAL NORDESTE I,Não,3.020200e+10,COORDENACAO REGIONAL NORDESTE I,CR-NE-I,"POLYGON ((-36.94514 -10.07113, -36.94377 -10.0..."
3,62,401,Água Preta/Inari,Apurinã,Pauini,AM,139763.6705,Regularizada,Tradicionalmente ocupada,None,COORDENACAO REGIONAL MÉDIO PURUS,Não,3.020200e+10,COORDENACAO REGIONAL MÉDIO PURUS,CR-MPUR,"POLYGON ((-67.29764 -7.33958, -67.29713 -7.338..."
4,3,501,Águas Belas,Pataxó,Prado,BA,1189.0649,Regularizada,Tradicionalmente ocupada,None,COORDENACAO REGIONAL SUL DA BAHIA,Não,3.020200e+10,COORDENACAO REGIONAL SUL DA BAHIA,CR-SB,"POLYGON ((-39.30549 -16.90608, -39.30501 -16.9..."


In [7]:
ti = ti[ti.geometry.intersects(cerrado.loc[0, 'geometry']).tolist()].reset_index(drop = True)

In [8]:
# Blending intersecting polygons into unique ones
ti = dissolve_interesct(ti)

In [9]:
# Create column
ti['pa_type'] = 'TI'
ti['pa_id'] = ['TI_' + str(i) for i in ti.index]

# Subset relevant columns
ti = ti[['pa_id','pa_type','geometry']]

In [10]:
ti.head()

,pa_id,pa_type,geometry
0,TI_0,TI,"POLYGON ((-55.19320 -23.15445, -55.19190 -23.1..."
1,TI_1,TI,"POLYGON ((-55.22764 -23.05481, -55.22314 -23.0..."
2,TI_2,TI,"POLYGON ((-47.82047 -5.97181, -47.82006 -5.971..."
3,TI_3,TI,"POLYGON ((-46.00619 -4.84956, -46.00557 -4.850..."
4,TI_4,TI,"POLYGON ((-51.89056 -14.15934, -51.89049 -14.1..."


In [11]:
# Calculate area in km2
ti = calculateArea_km2(ti)

In [12]:
# Save file 
ti.to_file('../../data/shapes/protected_areas/pa_terras-indigenas.shp')

## Terras indigenas + Protecao integral

In [13]:
dir_ = 'D:/carlota/projects/data/protected-areas_mma_SIRGAS2000/'
ucsei = gpd.read_file(dir_ + 'ucsei.shp')
ucsfi = gpd.read_file(dir_ + 'ucsfi.shp')
ucsmi = gpd.read_file(dir_ + 'ucsmi.shp')

In [14]:
ucsei = ucsei[ucsei.geometry.intersects(cerrado.loc[0, 'geometry']).tolist()].reset_index(drop = True)
ucsfi = ucsfi[ucsfi.geometry.intersects(cerrado.loc[0, 'geometry']).tolist()].reset_index(drop = True)
ucsmi = ucsmi[ucsmi.geometry.intersects(cerrado.loc[0, 'geometry']).tolist()].reset_index(drop = True)

In [15]:
ucsei = dissolve_interesct(ucsei)
ucsfi = dissolve_interesct(ucsfi)
ucsmi = dissolve_interesct(ucsmi)

In [16]:
# Create column
ucsei['pa_type'] = 'EI'
ucsei['pa_id'] = ['EI_' + str(i) for i in ucsei.index]

# Subset relevant columns
ucsei = ucsei[['pa_id','pa_type','geometry']]

# Create column
ucsfi['pa_type'] = 'FI'
ucsfi['pa_id'] = ['FI_' + str(i) for i in ucsfi.index]

# Subset relevant columns
ucsfi = ucsfi[['pa_id','pa_type','geometry']]

# Create column
ucsmi['pa_type'] = 'MI'
ucsmi['pa_id'] = ['MI_' + str(i) for i in ucsmi.index]

# Subset relevant columns
ucsmi = ucsmi[['pa_id','pa_type','geometry']]

In [53]:
# Concatenate all shapefiles (I know already that they are all in the same SIRGAS 2000 EPSG:4674 projection)
gdf = gpd.GeoDataFrame(pd.concat([ti, ucsei, ucsfi, ucsmi], ignore_index = True), crs = ti.crs)
gdf.head()

,pa_id,pa_type,geometry,area_km2
0,TI_0,TI,"POLYGON ((-55.19320 -23.15445, -55.19190 -23.1...",6.890110
1,TI_1,TI,"POLYGON ((-55.22764 -23.05481, -55.22314 -23.0...",24.275738
2,TI_2,TI,"POLYGON ((-47.82047 -5.97181, -47.82006 -5.971...",1432.583314
3,TI_3,TI,"POLYGON ((-46.00619 -4.84956, -46.00557 -4.850...",4174.371293
4,TI_4,TI,"POLYGON ((-51.89056 -14.15934, -51.89049 -14.1...",1803.885268


In [55]:
# Blend together intersecting geometries
gdf = dissolve_interesct(gdf)

In [56]:
# Create column
gdf['pa_type'] = 'TI_PI'
gdf['pa_id'] = ['TI_PI' + str(i) for i in gdf.index]

# Subset relevant columns
gdf = gdf[['pa_id','pa_type','geometry']]

In [57]:
# Calculate area in km2
gdf = calculateArea_km2(gdf)

In [58]:
gdf.head()

,pa_id,pa_type,geometry,area_km2
0,TI_PI0,TI_PI,"POLYGON ((-55.19320 -23.15445, -55.19190 -23.1...",6.890110
1,TI_PI1,TI_PI,"POLYGON ((-55.22764 -23.05481, -55.22314 -23.0...",24.275738
2,TI_PI2,TI_PI,"POLYGON ((-47.82047 -5.97181, -47.82006 -5.971...",1432.583314
3,TI_PI3,TI_PI,"POLYGON ((-46.00619 -4.84956, -46.00557 -4.850...",4174.371293
4,TI_PI4,TI_PI,"POLYGON ((-51.89056 -14.15934, -51.89049 -14.1...",1803.885268


In [59]:
gdf.to_file('../../data/shapes/protected_areas/pa_terras-indigenas_protecao-integral.shp')

## Uso sustentavel

In [61]:
dir_ = 'D:/carlota/projects/data/protected-areas_mma_SIRGAS2000/'
ucseu = gpd.read_file(dir_ + 'ucseu.shp')
ucsfu = gpd.read_file(dir_ + 'ucsfus.shp')
ucsmu = gpd.read_file(dir_ + 'ucsmu.shp')

In [62]:
ucseu = ucseu[ucseu.geometry.intersects(cerrado.loc[0, 'geometry']).tolist()].reset_index(drop = True)
ucsfu = ucsfu[ucsfu.geometry.intersects(cerrado.loc[0, 'geometry']).tolist()].reset_index(drop = True)
ucsmu = ucsmu[ucsmu.geometry.intersects(cerrado.loc[0, 'geometry']).tolist()].reset_index(drop = True)

In [63]:
ucseu = dissolve_interesct(ucseu)
ucsfu = dissolve_interesct(ucsfu)
ucsmu = dissolve_interesct(ucsmu)

In [64]:
# Create column
ucseu['pa_type'] = 'EU'
ucseu['pa_id'] = ['EU_' + str(i) for i in ucseu.index]

# Subset relevant columns
ucseu = ucseu[['pa_id','pa_type','geometry']]

# Create column
ucsfu['pa_type'] = 'FU'
ucsfu['pa_id'] = ['FU_' + str(i) for i in ucsfu.index]

# Subset relevant columns
ucsfu = ucsfu[['pa_id','pa_type','geometry']]

# Create column
ucsmu['pa_type'] = 'MU'
ucsmu['pa_id'] = ['MU_' + str(i) for i in ucsmu.index]

# Subset relevant columns
ucsmu = ucsmu[['pa_id','pa_type','geometry']]

In [65]:
# Concatenate all shapefiles (I know already that they are all in the same SIRGAS 2000 EPSG:4674 projection)
gdf = gpd.GeoDataFrame(pd.concat([ucseu, ucsfu, ucsmu], ignore_index = True), crs = ti.crs)
gdf.head()

,pa_id,pa_type,geometry
0,EU_0,EU,"MULTIPOLYGON (((-50.57350 -13.21929, -50.57208..."
1,EU_1,EU,"POLYGON ((-48.23114 -12.79940, -48.23111 -12.7..."
2,EU_2,EU,"POLYGON ((-48.95729 -23.26756, -48.95828 -23.2..."
3,EU_3,EU,"POLYGON ((-56.73498 -20.73780, -56.73714 -20.7..."
4,EU_4,EU,"MULTIPOLYGON (((-46.48686 -10.20326, -46.48665..."


In [66]:
# Blend together intersecting geometries
gdf = dissolve_interesct(gdf)

In [70]:
# Create column
gdf['pa_type'] = 'US'
gdf['pa_id'] = ['US_' + str(i) for i in gdf.index]

# Subset relevant columns
gdf = gdf[['pa_id','pa_type','geometry']]

In [71]:
# Calculate area in km2
gdf = calculateArea_km2(gdf)

In [72]:
gdf.head()

,pa_id,pa_type,geometry,area_km2
0,US_0,US,"MULTIPOLYGON (((-50.57208 -13.21940, -50.57178...",3816.948236
1,US_1,US,"POLYGON ((-48.23114 -12.79940, -48.23111 -12.7...",901.156930
2,US_2,US,"POLYGON ((-48.95729 -23.26756, -48.95828 -23.2...",2147.450588
3,US_3,US,"POLYGON ((-56.73498 -20.73780, -56.73714 -20.7...",0.117148
4,US_4,US,"MULTIPOLYGON (((-46.48665 -10.20330, -46.48644...",3106.728021


In [73]:
gdf.to_file('../../data/shapes/protected_areas/pa_uso-sustentavel.shp')

## All the areas

In [74]:
# Concatenate all shapefiles (I know already that they are all in the same SIRGAS 2000 EPSG:4674 projection)
gdf = gpd.GeoDataFrame(pd.concat([ti, ucsei, ucsfi, ucsmi, ucseu, ucsfu, ucsmu], ignore_index = True), crs = ti.crs)
gdf.head()

,pa_id,pa_type,geometry,area_km2
0,TI_0,TI,"POLYGON ((-55.19320 -23.15445, -55.19190 -23.1...",6.890110
1,TI_1,TI,"POLYGON ((-55.22764 -23.05481, -55.22314 -23.0...",24.275738
2,TI_2,TI,"POLYGON ((-47.82047 -5.97181, -47.82006 -5.971...",1432.583314
3,TI_3,TI,"POLYGON ((-46.00619 -4.84956, -46.00557 -4.850...",4174.371293
4,TI_4,TI,"POLYGON ((-51.89056 -14.15934, -51.89049 -14.1...",1803.885268


In [75]:
# Blend together intersecting geometries
gdf = dissolve_interesct(gdf)

In [76]:
# Create column
gdf['pa_type'] = 'PA'
gdf['pa_id'] = ['PA_' + str(i) for i in gdf.index]

# Subset relevant columns
gdf = gdf[['pa_id','pa_type','geometry']]

In [77]:
# Calculate area in km2
gdf = calculateArea_km2(gdf)

In [78]:
gdf.head()

,pa_id,pa_type,geometry,area_km2
0,PA_0,PA,"POLYGON ((-55.27099 -23.98518, -55.27769 -23.9...",6666.678918
1,PA_1,PA,"POLYGON ((-55.22764 -23.05481, -55.22314 -23.0...",24.275738
2,PA_2,PA,"POLYGON ((-47.82047 -5.97181, -47.82006 -5.971...",1432.583314
3,PA_3,PA,"POLYGON ((-46.00619 -4.84956, -46.00557 -4.850...",4174.371293
4,PA_4,PA,"POLYGON ((-51.89056 -14.15934, -51.89049 -14.1...",1803.885268


In [79]:
gdf.to_file('../../data/shapes/protected_areas/pa_all-areas.shp')